In [5]:
import random


class SensorOut:
    current_temp: float
    temp_list: list[float]

    def __init__(self, starting_temp: float = 19.0) -> None:
        self.current_temp = starting_temp
        self.temp_list = [self.current_temp]

    def timestep(self) -> None:
        temp_delta = random.uniform(-1, 1)
        self.current_temp += temp_delta
        self.temp_list.append(self.current_temp)

    def send_data(self):
        ...


class SensorIn:
    current_temp: float
    temp_list: list[float]
    temp_outside_factor: float
    temp_ac_factor: float

    def __init__(
        self,
        starting_temp: float = 19.0,
        temp_outside_factor: float = 0.5,
        temp_ac_factor: float = 0.5,
    ) -> None:
        self.current_temp = starting_temp
        self.temp_list = [self.current_temp]
        self.temp_outside_factor = temp_outside_factor
        self.temp_ac_factor = temp_ac_factor

    def timestep(self, temp_outside: float, temp_ac: float):
        self.current_temp = (
            self.current_temp
            + self.temp_outside_factor * temp_outside
            + self.temp_ac_factor * temp_ac
        ) / (1 + self.temp_outside_factor + self.temp_ac_factor)
        self.temp_list.append(self.current_temp)

    def send_data(self):
        ...

In [8]:
import numpy as np


def compute_ac_adjustment(
    inside_temps: list[float],
    outside_temps: list[float],
    ac_temp: float,
    goal_temp: float,
) -> float:
    inside_avg = np.array(inside_temps).mean()
    outside_avg = np.array(outside_temps).mean()

    if inside_avg < goal_temp:
        if outside_avg < goal_temp:
            ac_temp += 2
        else:
            ac_temp += 1
    else:
        if outside_avg < goal_temp:
            ac_temp -= 1
        else:
            ac_temp -= 2

    # Cap the max difference between ac and goal temperature
    threshold = 5
    if abs(goal_temp - ac_temp) > threshold:
        if goal_temp - ac_temp < 0:
            ac_temp = goal_temp + threshold
        else:
            ac_temp = goal_temp - threshold

    return ac_temp

In [9]:
TEMP_GOAL = 21.0
outside_sensor = SensorOut(starting_temp=19.0)
inside_sensor = SensorIn(starting_temp=19.0)
ac_temp = 20.0

for cloud_comm_round in range(10):
    for i in range(5):
        outside_sensor.timestep()
        inside_sensor.timestep(outside_sensor.current_temp, ac_temp)
        print(
            f"""
            Current Time Step: {i}
                Inside Temp: {inside_sensor.current_temp:.2f}      
                Outside Temp: {outside_sensor.current_temp:.2f}
                AC Temp: {ac_temp}
        """
        )
    ac_temp = compute_ac_adjustment(
        inside_sensor.temp_list, outside_sensor.temp_list, ac_temp, TEMP_GOAL
    )
    print(f"Talking to the cloud new ac temp is {ac_temp}")
    inside_sensor.temp_list = []
    outside_sensor.temp_list = []


            Current Time Step: 0
                Inside Temp: 19.01      
                Outside Temp: 18.02
                AC Temp: 20.0
        

            Current Time Step: 1
                Inside Temp: 19.09      
                Outside Temp: 18.36
                AC Temp: 20.0
        

            Current Time Step: 2
                Inside Temp: 19.01      
                Outside Temp: 17.86
                AC Temp: 20.0
        

            Current Time Step: 3
                Inside Temp: 18.86      
                Outside Temp: 17.42
                AC Temp: 20.0
        

            Current Time Step: 4
                Inside Temp: 18.73      
                Outside Temp: 17.18
                AC Temp: 20.0
        
Talking to the cloud new ac temp is 22.0

            Current Time Step: 0
                Inside Temp: 19.26      
                Outside Temp: 17.58
                AC Temp: 22.0
        

            Current Time Step: 1
                Inside Te

In [42]:
from google.oauth2 import service_account
from google.auth.transport.requests import AuthorizedSession


url = "https://europe-west1-fog-computing-2024.cloudfunctions.net/compute_ac_adjustment"

creds = service_account.IDTokenCredentials.from_service_account_file(
    "/Users/felixzailskas/.config/gcloud/fog-computing-2024-4d89feedd16b.json",
    target_audience=url,
)

body = {
    "inside_temps": [20.5, 20.5],
    "outside_temps": [20.5, 20.5],
    "ac_temp": 22.0,
    "goal_temp": 21.0,
}

headers = {
    "Content-Type": "application/json",
    "Accept": "application/json",
}

authed_session = AuthorizedSession(creds)

# make authenticated request and print the response, status_code
resp = authed_session.post(url, json=body, headers=headers)
print(resp.status_code)
print(resp.text)

200
{"ac_temp":24.0}

